In [1]:
import pathlib
import textwrap
import time

import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

import re
import json
import uuid
import pandas as pd

In [2]:
# Used to securely store your API key
from google.colab import userdata

# Or use `os.getenv('GOOGLE_API_KEY')` to fetch an environment variable.
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

In [3]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-1.5-pro-latest
models/gemini-pro
models/gemini-pro-vision


In [4]:
model = genai.GenerativeModel('gemini-1.0-pro')

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import pandas as pd

file_path = '/content/drive/My Drive/598/code/Self-Reward-RLHF/generated/responses.jsonl'
df = pd.read_json(file_path, lines=True)
df

,prompt_id,prompt,completion
0,ff8295a6-9b2b-4154-a314-d1cf924c183f,"Sorry for being so rude, the dataset in questi...",{'0': 'I apologize if my previous response see...
1,26b3ef11-afb3-4ed4-977e-ef23da46adbe,Improve the writing in the following email:\n\...,{'0': 'I would suggest the following improveme...
2,e24617e5-ceff-45dc-a027-16b43935427d,Can you please explain how to create a new fol...,"{'0': 'Sure, here is a step-by-step guide on h..."
3,eee58322-7042-4494-b825-9a88856babe2,Create a 8 sentence story outline for a book a...,{'0': 'The story revolves around two individua...
4,4dae55d7-8209-4226-9a82-fb973d767966,"Ok, sure. I would like you to talk me through ...",{'0': 'Vite is an automated build system for R...
...,...,...,...
996,eab28afd-79e8-4292-8a41-f3ed344a4e54,Thanks. What are some examples for popular app...,{'0': 'Some popular applications that use elec...
997,fd187e3f-7333-4338-bbab-c6e0cf28397b,How do I style my HTML page using CSS?,{'0': 'Styling your HTML page using CSS is a g...
998,2b573f77-ca80-4a33-aab5-a27e5af59671,what are the top 10 tallest buildings in the w...,{'0': 'Here are the top 10 tallest buildings i...
999,e602946b-009b-497e-9c49-db789853d4e9,Now make a timeline table of all of it and wri...,{'0': 'Here's a timeline table that summarizes...


In [7]:
srlm_prompt = """Review the user’s question and the corresponding response using the additive 5-point
    scoring system described below.

    The user's question is between <question> and </question>
    The response of the AI Assistant is between <response> and </response>

    Points are accumulated based on the satisfaction of each
    criterion:
    - Add 1 point if the response is relevant and provides some information related to
    the user’s inquiry, even if it is incomplete or contains some irrelevant content.
    - Add another point if the response addresses a substantial portion of the user’s question,
    but does not completely resolve the query or provide a direct answer.
    - Award a third point if the response answers the basic elements of the user’s question in a
    useful way, regardless of whether it seems to have been written by an AI Assistant or if it
    has elements typically found in blogs or search results.
    - Grant a fourth point if the response is clearly written from an AI Assistant’s perspective,
    addressing the user’s question directly and comprehensively, and is well-organized and
    helpful, even if there is slight room for improvement in clarity, conciseness or focus.
    - Bestow a fifth point for a response that is impeccably tailored to the user’s question
    by an AI Assistant, without extraneous information, reflecting expert knowledge, and
    demonstrating a high-quality, engaging, and insightful answer.
    - If the response repeats itself or is not concise and to the point, score the response 0.

    <question>{prompt}</question>
    <response>{response}</response>

    After examining the user’s instruction and the response:
    - output the score of the evaluation using this exact format: "score: <total points>", where <total points> is between 0 and 5
    - Briefly justify your total score, up to 100 words.
    """

In [9]:
pattern = r"[Ss]core: ([0-5])"
results = []
for index, row in df.iterrows():
    prompt_id = row['prompt_id']
    prompt = row['prompt']
    completion = row['completion']
    buf = {}
    print("-============================="+ str(index))
    for i in range(4):
        print("-------------------------")
        buf[str(i)] = {
            'content':completion[str(i)],
            'score':0,
            "reasoning":""
        }
        llm_as_a_judge_prompt = srlm_prompt.format(prompt=prompt,response=completion[str(i)])

        try:
            answer = model.generate_content(llm_as_a_judge_prompt)
        except Exception as e:
            print(f"发生错误：{e}")
            print("等待5秒后重试...")
            time.sleep(5)
            answer = model.generate_content(llm_as_a_judge_prompt)
            print("程序继续执行")

        matches = re.findall(pattern, answer.text)
        generated_score = int(matches[0]) if matches else -1
        print("Found Score: ", generated_score)
        buf[str(i)]['score'] = generated_score
        buf[str(i)]['reasoning'] = answer.text
        time.sleep(1.3)

    results.append({
        "prompt_id": prompt_id,
        "prompt": prompt,
        "completion": buf,
    })

-=============================0
-------------------------
Found Score:  4
-------------------------
Found Score:  4
-------------------------
Found Score:  4
-------------------------
Found Score:  2
-=============================1
-------------------------
Found Score:  5
-------------------------
Found Score:  0
-------------------------
Found Score:  2
-------------------------
Found Score:  3
-=============================2
-------------------------
Found Score:  5
-------------------------
Found Score:  5
-------------------------
Found Score:  5
-------------------------
Found Score:  5
-=============================3
-------------------------
Found Score:  4
-------------------------
Found Score:  5
-------------------------
Found Score:  4
-------------------------
Found Score:  4
-=============================4
-------------------------
Found Score:  4
-------------------------
Found Score:  3
-------------------------
Found Score:  2
-------------------------
Found Score:  3


KeyboardInterrupt: 

In [ ]:
df_results = pd.DataFrame(results)
df_results.to_json('/content/drive/My Drive/598/code/Self-Reward-RLHF/generated/scores.jsonl', orient='records', lines=True, mode = 'w')

In [ ]:
results

In [ ]:
# pattern = r"[Ss]core: ([0-5])"
# start = 16
# for index, row in df.iterrows():
#     if index < start:
#       continue
#     prompt_id = row['prompt_id']
#     prompt = row['prompt']
#     completion = row['completion']
#     buf = {}
#     print("-============================="+ str(index))
#     for i in range(4):
#         print("-------------------------")
#         buf[str(i)] = {
#             'content':completion[str(i)],
#             'score':0,
#             "reasoning":""
#         }
#         llm_as_a_judge_prompt = srlm_prompt.format(prompt=prompt,response=completion[str(i)])

#         try:
#             answer = model.generate_content(llm_as_a_judge_prompt)
#         except Exception as e:
#             print(f"发生错误：{e}")
#             print("等待5秒后重试...")
#             time.sleep(5)
#             answer = model.generate_content(llm_as_a_judge_prompt)
#             print("程序继续执行")

#         matches = re.findall(pattern, answer.text)
#         generated_score = int(matches[0]) if matches else -1
#         print("Found Score: ", generated_score)
#         buf[str(i)]['score'] = generated_score
#         buf[str(i)]['reasoning'] = answer.text
#         time.sleep(1.3)

#     results.append({
#         "prompt_id": prompt_id,
#         "prompt": prompt,
#         "completion": buf,
#     })